In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import pdb
import copy
from scipy import sparse
import anndata
import cerberus
import scipy
from pandarallel import pandarallel


p = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

In [2]:
h5 = '../cerberus_annot.h5'
h5_annot = 'cerberus_annot_triplets.h5'
h5_annot_mouse = '../../../mouse/lr_bulk/triplets/cerberus_annot_triplets.h5'
swan_file = '../swan/swan.p'
mouse_swan_file = '../../../mouse/lr_bulk/swan/swan.p'
ab = '../../talon/human_talon_abundance.tsv'
mouse_filt_ab = '../../../mouse/lr_bulk/cerberus/cerberus_filtered_abundance.tsv'
filt_ab = '../cerberus_filtered_abundance.tsv'
obs_col = 'sample'
min_tpm = 1
major_set = '../swan/isos_sample_gene_90.tsv'
major_set_mouse = '../swan/isos_sample_gene_90.tsv'
mm_tissues = get_mouse_match_samples()

In [3]:
ca = cerberus.read(h5_annot)


In [4]:
# calculate centroid of each gene based on sample_det
# and compute mean number of isoforms expressed
source = 'sample_det'
df = ca.triplets.loc[ca.triplets.source == source].copy(deep=True)
df = cerberus.compute_simplex_coords(df, 'splicing_ratio')
keep_cols = ['gname', 'gid', 'tss_ratio', 'tes_ratio', 'spl_ratio', 'n_iso']
df = df[keep_cols]
df = df.groupby(['gname', 'gid']).mean().reset_index()

In [9]:
def do_go_thing(df, iso_threshes):
    dbs = ['GO_Biological_Process_2021',
           'GO_Cellular_Component_2021',
           'GO_Molecular_Function_2021',
           'KEGG_2021_Human']
    bm = gp.parser.Biomart()
    datasets = bm.get_datasets(mart='ENSEMBL_MART_ENSEMBL')
    datasets.loc[datasets.Description.str.contains('Human')]
    
    for iso_thresh in iso_threshes:
        print(iso_thresh)
        odir = '{}_isos_genes_GO'.format(iso_thresh)
        desc = odir
        gids = df.loc[df.n_iso >= iso_thresh, 'gid'].tolist()
        gids = bm.query(dataset='hsapiens_gene_ensembl',
               attributes=['ensembl_gene_id', 'external_gene_name'],
               filters={'ensembl_gene_id': gids})
        gids = gids.loc[~gids.external_gene_name.isna()]
        gnames = gids.external_gene_name.squeeze().str.strip().tolist()
        go = gp.enrichr(gene_list=gnames,
                        gene_sets=dbs,
                        organism='Human',
                        description=desc,
                        outdir=odir,
                        cutoff=0.5)

In [10]:
do_go_thing(df, [5,10])

5
10
